In [1]:
%pip install openpyxl pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
# 버스 정류장 데이터를 불러온다
station_df = pd.read_csv('국토교통부_전국 버스정류장 위치정보_20231016.csv', encoding='cp949')

In [4]:
station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204209 entries, 0 to 204208
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   정류장번호    204209 non-null  object 
 1   정류장명     204207 non-null  object 
 2   위도       204207 non-null  float64
 3   경도       204207 non-null  float64
 4   정보수집일    204209 non-null  object 
 5   모바일단축번호  187350 non-null  float64
 6   도시코드     204209 non-null  int64  
 7   도시명      204209 non-null  object 
 8   관리도시명    204209 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 14.0+ MB


In [5]:
# NA 값이 존재하는지 확인한다
for col in station_df.columns:
    if sum(station_df[col].isna()):
        print(col)

정류장명
위도
경도
모바일단축번호


In [6]:
station_df[station_df['위도'].isna()]

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명
133400,MYB6900,NaN,NaN,NaN,2023-10-16,NaN,38080,경상남도 밀양시,밀양
133401,MYB6901,NaN,NaN,NaN,2023-10-16,NaN,38080,경상남도 밀양시,밀양


In [7]:
station_df[station_df['경도'].isna()]

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명
133400,MYB6900,NaN,NaN,NaN,2023-10-16,NaN,38080,경상남도 밀양시,밀양
133401,MYB6901,NaN,NaN,NaN,2023-10-16,NaN,38080,경상남도 밀양시,밀양


In [8]:
# 위도와 경도가 NaN 값을 가지고 있다면, 해당 row를 삭제한다
station_df.dropna(subset=['위도', '경도'], inplace=True)

# NA 값이 존재하는지 확인한다
for col in station_df.columns:
    if sum(station_df[col].isna()):
        print(col)

모바일단축번호


In [9]:
station_df['정류장번호'].nunique() == len(station_df)

True

In [10]:
# 컬럼명을 영문으로 변경한다
station_df.rename(columns={
    '정류장번호': 'node_id',
    '정류장명': 'node_name',
    '위도': 'latitude',
    '경도': 'longitude',
    '정보수집일': 'collectd_time',
    '모바일단축번호': 'mobile_id',
    '도시코드': 'city_code',
    '도시명': 'city_name',
    '관리도시명': 'admin_name'
}, inplace=True)

# mobile_id에 na 값 여부를 확인한다
sum(station_df['mobile_id'].isna())

# NA 값은 0으로 채우고, 데이터 타입을 Int형으로 변환한다
station_df['mobile_id'] = station_df['mobile_id'].fillna(0)
station_df['mobile_id'] = station_df['mobile_id'].astype('int')
# 정류장명 데이터 타입을 str형으로 변환한다
station_df['node_name'] = station_df['node_name'].astype(str)

# 데이터 중에 위도와 경도가 값이 바뀐 데이터가 있으므로, 올바르게 변경해준다
idx = station_df.index[station_df['node_id'] == 'JEB406002082'].tolist()[0]
station_df.loc[idx, ['latitude', 'longitude']] = station_df.loc[idx, ['longitude', 'latitude']].values

# UTF-8 encoding으로 다시 저장한다
station_filename = 'bus_station.csv'
station_df.to_csv(station_filename, encoding='utf-8', index=False)

In [11]:
station_df = pd.read_csv(station_filename)

In [12]:
station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204207 entries, 0 to 204206
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   node_id        204207 non-null  object 
 1   node_name      204207 non-null  object 
 2   latitude       204207 non-null  float64
 3   longitude      204207 non-null  float64
 4   collectd_time  204207 non-null  object 
 5   mobile_id      204207 non-null  int64  
 6   city_code      204207 non-null  int64  
 7   city_name      204207 non-null  object 
 8   admin_name     204207 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 14.0+ MB


In [13]:
station_df.head(3)

,node_id,node_name,latitude,longitude,collectd_time,mobile_id,city_code,city_name,admin_name
0,ADB354000076,신덕1리,36.524325,128.830926,2023-10-16,540076,37040,경상북도 안동시,안동
1,ADB354000077,나천,36.484444,128.876390,2023-10-16,540077,37040,경상북도 안동시,안동
2,ADB354000078,골삽실,36.462248,128.887606,2023-10-16,540078,37040,경상북도 안동시,안동


In [14]:
station_df[station_df['mobile_id'] == 7433]

,node_id,node_name,latitude,longitude,collectd_time,mobile_id,city_code,city_name,admin_name
62658,GGB106000438,서울의료원후문,37.613750,127.098200,2023-10-16,7433,11,서울특별시,경기
71884,GGB206000458,우성상가,37.377517,127.132267,2023-10-16,7433,31020,경기도 성남시,경기


In [15]:
station_df[station_df['latitude'] == 126.415368]

,node_id,node_name,latitude,longitude,collectd_time,mobile_id,city_code,city_name,admin_name


---

In [16]:
route_df = pd.read_excel('서울시버스노선별정류소정보(20231215).xlsx', engine='openpyxl')

In [17]:
route_df.head(3)

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표
0,100100288,5714,1,213000455,38629,광명공영차고지기점,126.872566,37.458065
1,116000006,703,1,219000691,36022,경남아너스빌.탄현역,126.764273,37.693393
2,100100398,9703,1,219000691,36022,경남아너스빌.탄현역,126.764273,37.693393


In [18]:
# NA 값이 존재하는지 확인한다
for col in route_df.columns:
    if sum(route_df[col].isna()):
        print(col)

In [19]:
# 컬럼명을 영문으로 통일한다
route_df.rename(columns={
    'ROUTE_ID': 'route_id',
    '노선명': 'route_name',
    '순번': 'route_order',
    'NODE_ID': 'node_id',
    'ARS_ID': 'ars_id',
    '정류소명': 'station_name',
    'X좌표': 'longitude',
    'Y좌표': 'latitude'
}, inplace=True)

route_filename = 'bus_route.csv'
route_df.to_csv(route_filename, encoding='utf-8', index=False)

In [20]:
route_df = pd.read_csv(route_filename)

In [21]:
route_df.head(3)

,route_id,route_name,route_order,node_id,ars_id,station_name,longitude,latitude
0,100100288,5714,1,213000455,38629,광명공영차고지기점,126.872566,37.458065
1,116000006,703,1,219000691,36022,경남아너스빌.탄현역,126.764273,37.693393
2,100100398,9703,1,219000691,36022,경남아너스빌.탄현역,126.764273,37.693393


In [22]:
route_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46654 entries, 0 to 46653
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   route_id      46654 non-null  int64  
 1   route_name    46654 non-null  object 
 2   route_order   46654 non-null  int64  
 3   node_id       46654 non-null  int64  
 4   ars_id        46654 non-null  int64  
 5   station_name  46654 non-null  object 
 6   longitude     46654 non-null  float64
 7   latitude      46654 non-null  float64
dtypes: float64(2), int64(4), object(2)
memory usage: 2.8+ MB


In [23]:
route_df['route_id'].nunique() == len(route_df)

False